In [1]:
#Start of GA:
from random import seed
from random import choices
import random
import numpy as np 
import pandas as pd
from wptherml.wpml import multilayer
from deap import base, creator, tools, algorithms

#multi_thick generates a random number to represent thicknesses 
#has a top limit of 2000nm for each layer
def multi_thick():
    sys_random = random.SystemRandom()
    seq =  np.linspace(100e-9, 2000e-9, 191)

#randomly choose 4 numbers from the seq list: dealing with 4 layers only for now
    for i in range(4):
        select = sys_random.choices(seq, k=4)
    return select

#multi_mat generates 4 random numbers to represent different materials in the structure
def multi_mat():
    
    sys_random = random.SystemRandom()
    
    randomintlist = sys_random.sample(range(0, 4), 4)
    return randomintlist

#maximise fitness func initialisation
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax) 
toolbox = base.Toolbox()     
toolbox.register('multi_thick', multi_thick) 
toolbox.register('multi_mat', multi_mat) 
toolbox.register('individual', tools.initCycle, creator.Individual, (toolbox.multi_thick, toolbox.multi_mat), n=1)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

#no. of individuals per pop
N_pop = 80

population = toolbox.population(n=N_pop) 

#fitness func of GA
def evaluate(individual):    

    structure = {

        'Material_List': ['Air', 'HfO2','CaF', 'SiO2', 'HfO2', 'Ag', 'Air'],
        'Thickness_List': [0, 230e-9, 485e-9, 688e-9, 13e-9, 3e-6, 0],
        'Lambda_List': [300e-9, 13000e-9, 1000]
     
        }
    
    cool_ml = multilayer(structure)
    
    #this loop is for updating thickness to .d array
    # [i+1] as i dont want to touch first air thickness
    for i in range(4):
        cool_ml.d[i+1]=individual[0][i]  
    
    #need another loop for updating materials
    #initialise dict for converting number array to named mat list
    dic = {0:'CaF',1: 'Al2O3', 2: 'Si3N4', 3: 'SiO2'}
    named_matlist = []
    
    #for loop below converts number into named material list
    #need to consider the position of materials coz air layers and Ag layer is unchanged
    for i in individual[1]:
        named_matlist.append(dic[i])
    
    named_matlist.insert(0, 'Air')
    named_matlist.insert(5, 'Ag')
    #need to inserting air into last layer according to dev 
    named_matlist.insert(6, 'Air')
    
    structure['Material_List'] = named_matlist
    
    #need to update fresnal and thermal emiss before computing power
    
    cool_ml = multilayer(structure)
    cool_ml.fresnel()
    
    #for 1000 lambda:
    #alpha calc
    avg_emiss_alpha = np.sum(cool_ml.emissivity_array[0:174])/174
    
    #epsilon calc
    avg_emiss_epsilon = np.sum(cool_ml.emissivity_array[606:1000])/394 

    #returns based on fitness function
    return avg_emiss_epsilon-avg_emiss_alpha,

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=4)



In [3]:
import pandas as pd
import timeit
 
def main():
    
    start = timeit.default_timer()
    
    pop = toolbox.population(n=N_pop)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        
        ind.fitness.values = fit
        
    # CXPB = crossover rate

    # MUTPB = mutation rate
    
    CXPB, MUTPB = 0.8, 0.05
    
    # Extracting fitnesses 
    fits = [ind.fitness.values[0] for ind in pop]
    
    # Track no. of generations
    g = 0
    
    output_list = []
    
    # Start of evolution
    while g < 50:
    
        g = g + 1
        print("-- Generation %i --" % g)
        
        # Select next gen individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
        # Crossover and mutation
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values
            
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        results = max(fits), mean
        print(results)
        
        output_list.append(results)
        
        if g == 50:
            for x in pop:
                print(x)
                print(toolbox.evaluate(x))
            
            best = pop[np.argmax([toolbox.evaluate(x) for x in pop])]
            print(best)
            
    #test additional code        
    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    
    for i in output_list:
        print(i)
    
    stop = timeit.default_timer()
    print('Time: ', stop - start) 
    return 

if __name__ == "__main__":
    main()


  Gradient will be taken with respect to all layers! 


OSError: C:\Users\44773\anaconda3\envs\mltech\Lib\site-packages\wptherml\datalib\CaF.txt not found.